# Run Local Race Classification Prediction

This notebook loads the local Keras `.h5` race-classification model and performs predictions on images from your machine. Optionally, you can use YOLOv8 to first detect people/faces and then classify the cropped regions.

- Update `MODEL_PATH` if your `.h5` name differs
- Update `TEST_IMAGE_PATH` to any local image file


In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Configuration
# Adjust these if your filenames/paths differ
MODEL_PATH = "/Users/jeron/projects/Race-Classification/Race Classification Model.h5"
TEST_IMAGE_PATH = "/Users/jeron/Pictures/sample.jpg"  # Change to a local image file
CLASS_NAMES = ["Black", "East Asian", "White"]  # match the training setup

print(tf.__version__)
print(f"Model path exists: {os.path.exists(MODEL_PATH)}")
print(f"Test image exists: {os.path.exists(TEST_IMAGE_PATH)}")


ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Load Keras model (.h5)
assert os.path.exists(MODEL_PATH), f"Model not found at {MODEL_PATH}"
model = tf.keras.models.load_model(MODEL_PATH)
model.summary()


In [ ]:
from typing import Tuple

def preprocess_image(img_path: str, target_size: Tuple[int, int] = (224, 224)) -> np.ndarray:
    """Load and preprocess image for the classifier."""
    img = image.load_img(img_path, target_size=target_size)
    arr = image.img_to_array(img)
    arr = np.expand_dims(arr, axis=0)
    arr = arr / 255.0
    return arr


def predict_image(img_path: str) -> Tuple[str, np.ndarray]:
    arr = preprocess_image(img_path)
    preds = model.predict(arr)
    idx = int(np.argmax(preds, axis=-1)[0])
    return CLASS_NAMES[idx], preds[0]



In [ ]:
# Demo: predict on a single image
assert os.path.exists(TEST_IMAGE_PATH), f"Test image not found: {TEST_IMAGE_PATH}"
label, probs = predict_image(TEST_IMAGE_PATH)
print("Predicted:", label)
print("Probabilities:", {name: float(p) for name, p in zip(CLASS_NAMES, probs)})

img = image.load_img(TEST_IMAGE_PATH)
plt.imshow(img)
plt.axis('off')
plt.title(f"Predicted: {label}")
plt.show()


In [ ]:
# Optional: YOLOv8 detection → classify crops
# pip install ultralytics opencv-python  # run in your environment if needed
from ultralytics import YOLO
import cv2

YOLO_MODEL = "yolov8m.pt"  # change to 'yolov8n.pt' for faster CPU inference

# Path to an image with people/faces
DETECT_IMAGE_PATH = TEST_IMAGE_PATH  # reuse, or set another file path
assert os.path.exists(DETECT_IMAGE_PATH), f"Image not found: {DETECT_IMAGE_PATH}"

# Load detector
yolo = YOLO(YOLO_MODEL)

# Read and run detection
bgr = cv2.imread(DETECT_IMAGE_PATH)
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
res = yolo(rgb)[0]

# Visualize detections
plt.figure()
plt.imshow(res.plot())
plt.axis('off')
plt.title("YOLO detections")
plt.show()

# Classify each detected person region
predictions = []
for box, cls_id in zip(res.boxes.xyxy.cpu().numpy(), res.boxes.cls.cpu().numpy().astype(int)):
    x1, y1, x2, y2 = map(int, box[:4])
    crop = rgb[y1:y2, x1:x2]
    if crop.size == 0:
        continue
    # Save to temp and reuse same preprocessing
    crop_path = "/tmp/yolo_crop.jpg"
    cv2.imwrite(crop_path, cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))
    label, probs = predict_image(crop_path)
    predictions.append(((x1, y1, x2, y2), label, probs))

# Show crops with predicted labels
for (x1, y1, x2, y2), label, _ in predictions:
    plt.figure()
    plt.imshow(rgb[y1:y2, x1:x2])
    plt.axis('off')
    plt.title(f"Predicted: {label}")
    plt.show()
